In [1]:
import pandas as pd
import numpy as np 
import matplotlib.pyplot as plt
import seaborn as sns
import tensorflow as tf 
from sklearn.model_selection import train_test_split
from tensorflow.keras import layers
import keras as keras
from keras import models
from keras import layers
from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation
from keras.optimizers import SGD
from keras.utils.np_utils import to_categorical
from keras.wrappers.scikit_learn import KerasClassifier
from sklearn.model_selection import cross_val_score
from sklearn.neural_network import MLPClassifier
from keras import metrics
from sklearn.metrics import mean_squared_error
from math import sqrt
from sklearn.preprocessing import LabelBinarizer
from sklearn.model_selection import StratifiedKFold
from sklearn.model_selection import cross_val_predict
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score
from copy import deepcopy

Using TensorFlow backend.


In [2]:
red = pd.read_csv('data/winequality-red1.csv')
white = pd.read_csv('data/winequality-white1.csv')

In [3]:
red['quality'].replace(to_replace=[0,1,2,3,4,5], value=1, inplace=True)
red['quality'].replace(to_replace=[6], value=2, inplace=True)
red['quality'].replace(to_replace=[7,8,9,10], value=3, inplace=True)
X_red = red[
['fixed acidity',
 'volatile acidity',
 'citric acid',
 'residual sugar',
 'chlorides',
 'free sulfur dioxide',
 'total sulfur dioxide',
 'density',
 'pH',
 'sulphates',
 'alcohol']]
Y_red = red[['quality']]
white['quality'].replace(to_replace=[0,1,2,3,4,5], value=1, inplace=True)
white['quality'].replace(to_replace=[6], value=2, inplace=True)
white['quality'].replace(to_replace=[7,8,9,10], value=3, inplace=True)
X_white = white[
['fixed acidity',
 'volatile acidity',
 'citric acid',
 'residual sugar',
 'chlorides',
 'free sulfur dioxide',
 'total sulfur dioxide',
 'density',
 'pH',
 'sulphates',
 'alcohol']]

Y_white = white[['quality']]
#X_red = X_red.values
Y_red = Y_red.values
#X_white = X_white.values
Y_white = Y_white.values
number_of_features = 11

In [4]:
def rsme(targets, outputs):
    return tf.sqrt(tf.reduce_mean(tf.square(tf.subtract(targets, outputs))))  

cv = StratifiedKFold(n_splits=10, shuffle=True, random_state=42)

# Create function returning a compiled network
def create_network():
    network = models.Sequential()
    #network.add(layers.BatchNormalization())
    network.add(layers.Dense(units=64, activation='linear'))
    #network.add(layers.Dense(units=9, activation = 'relu'))
    #network.add(layers.Dense(units=6, activation = 'relu'))
    #network.add(layers.Dense(units=18, activation = 'relu'))
    #network.add(layers.Dense(3, activation='softmax'))

    #network.compile(loss='binary_crossentropy', # Cross-entropy
   #                 optimizer='rmsprop', # Root Mean Square Propagation
    #                metrics=['accuracy']) # Accuracy performance metric
    network.compile(loss='sparse_categorical_crossentropy',
              optimizer='adam',
              metrics=['accuracy', rsme])
    
    # Return compiled network
    return network


neural_network = KerasClassifier(build_fn=create_network, 
                                     epochs=10, 
                                     batch_size=100, 
                                     verbose=0)

In [5]:
def startStatistic(neural_network, X_red, Y_red, X_white, Y_white):
    X_red = X_red.values
    print("Red")
    prediction_red = cross_val_predict(neural_network, X_red, Y_red, cv=cv) 
    print(classification_report(Y_red, prediction_red))
    print("RSME red")
    print(sqrt(np.mean((Y_red-prediction_red)*(Y_red-prediction_red))))
    X_white = X_white.values
    print("White")
    prediction_white = cross_val_predict(neural_network, X_white, Y_white, cv=cv) 
    print(classification_report(Y_white, prediction_white))
    print('RSME white')
    print(sqrt(np.mean((Y_white-prediction_white)*(Y_white-prediction_white))))

## One Layer

In [6]:
# Create function returning a compiled network
def create_network_linear():
    network = models.Sequential()
    network.add(layers.Dense(units=64, activation='linear'))

    network.compile(loss='sparse_categorical_crossentropy',
              optimizer='adam',
              metrics=['accuracy', rsme])
    
    # Return compiled network
    return network

neural_network = KerasClassifier(build_fn=create_network_linear, 
                                     epochs=10, 
                                     batch_size=100, 
                                     verbose=0)
print("One Layer Linear")
startStatistic(neural_network, X_red, Y_red, X_white, Y_white)

One Layer Linear
Red


IndexError: index 15 is out of bounds for axis 1 with size 3

In [18]:
network = models.Sequential()
network.add(layers.Dense(units=64, activation='sigmoid'))
network.compile(loss='sparse_categorical_crossentropy',
              optimizer='adam',
              metrics=['accuracy', rsme])
neural_network = KerasClassifier(build_fn=network, 
                                     epochs=10, 
                                     batch_size=100, 
                                     verbose=0) 
print("One Layer Sigmoid")
startStatistic(neural_network, X_red, X_white)

White
              precision    recall  f1-score   support

           1       0.65      0.63      0.64      1640
           2       0.55      0.66      0.60      2198
           3       0.59      0.37      0.46      1060

   micro avg       0.59      0.59      0.59      4898
   macro avg       0.60      0.56      0.57      4898
weighted avg       0.59      0.59      0.58      4898

RSME white
0.982920202845265


In [19]:
print("BN, One layer Relu with 64 nodes, Relu 9 nodes, Relu 6, Relu18 and Softmax")
print("Red")
prediction_red = cross_val_predict(neural_network, X_red, Y_red, cv=cv) 
print(classification_report(Y_red, prediction_red))
print("White")
prediction_white = cross_val_predict(neural_network, X_white, Y_white, cv=cv) 
print(classification_report(Y_white, prediction_white))
print("RSME red")
print(sqrt(np.mean((Y_red-prediction_red)*(Y_red-prediction_red))))
print('RSME white')
print(sqrt(np.mean((Y_white-prediction_white)*(Y_white-prediction_white))))

BN, One layer Relu with 64 nodes, Relu 9 nodes, Relu 6, Relu18 and Softmax
Red
              precision    recall  f1-score   support

           1       0.68      0.79      0.73       744
           2       0.53      0.56      0.54       638
           3       0.55      0.13      0.21       217

   micro avg       0.61      0.61      0.61      1599
   macro avg       0.58      0.49      0.49      1599
weighted avg       0.60      0.61      0.58      1599

White
              precision    recall  f1-score   support

           1       0.66      0.62      0.64      1640
           2       0.56      0.64      0.60      2198
           3       0.59      0.45      0.51      1060

   micro avg       0.60      0.60      0.60      4898
   macro avg       0.60      0.57      0.58      4898
weighted avg       0.60      0.60      0.59      4898

RSME red
0.9161692350251881
RSME white
0.9979839959440506


In [9]:
print("BN, One layer Sigmoid with 64 nodes, relu 9 nodes and Softmax")
print("Red")
prediction_red = cross_val_predict(neural_network, X_red, Y_red, cv=cv) 
print(classification_report(Y_red, prediction_red))
print("White")
prediction_white = cross_val_predict(neural_network, X_white, Y_white, cv=cv) 
print(classification_report(Y_white, prediction_white))
print("RSME red")
print(sqrt(np.mean((Y_red-prediction_red)*(Y_red-prediction_red))))
print('RSME white')
print(sqrt(np.mean((Y_white-prediction_white)*(Y_white-prediction_white))))

BN, One layer Sigmoid with 64 nodes, relu 9 nodes and Softmax
Red
              precision    recall  f1-score   support

           1       0.65      0.83      0.73       744
           2       0.52      0.52      0.52       638
           3       0.60      0.07      0.12       217

   micro avg       0.60      0.60      0.60      1599
   macro avg       0.59      0.47      0.46      1599
weighted avg       0.60      0.60      0.57      1599

White
              precision    recall  f1-score   support

           1       0.63      0.58      0.60      1640
           2       0.53      0.67      0.59      2198
           3       0.58      0.34      0.42      1060

   micro avg       0.57      0.57      0.57      4898
   macro avg       0.58      0.53      0.54      4898
weighted avg       0.57      0.57      0.56      4898

RSME red
0.9100337580554019
RSME white
0.9710443331885646


In [12]:
print("BN, One layer Relu with 64 nodes, relu 9 nodes and Softmax")
print("Red")
prediction_red = cross_val_predict(neural_network, X_red, Y_red, cv=cv) 
print(classification_report(Y_red, prediction_red))
print("White")
prediction_white = cross_val_predict(neural_network, X_white, Y_white, cv=cv) 
print(classification_report(Y_white, prediction_white))
print("RSME red")
print(sqrt(np.mean((Y_red-prediction_red)*(Y_red-prediction_red))))
print('RSME white')
print(sqrt(np.mean((Y_white-prediction_white)*(Y_white-prediction_white))))

BN, One layer Relu with 64 nodes, relu 9 nodes and Softmax
Red
              precision    recall  f1-score   support

           1       0.69      0.81      0.74       744
           2       0.55      0.56      0.56       638
           3       0.59      0.20      0.30       217

   micro avg       0.63      0.63      0.63      1599
   macro avg       0.61      0.52      0.53      1599
weighted avg       0.62      0.63      0.61      1599

White
              precision    recall  f1-score   support

           1       0.65      0.65      0.65      1640
           2       0.56      0.64      0.60      2198
           3       0.60      0.43      0.50      1060

   micro avg       0.59      0.59      0.59      4898
   macro avg       0.60      0.57      0.58      4898
weighted avg       0.60      0.59      0.59      4898

RSME red
0.9284291328541032
RSME white
0.9975910531370398


In [13]:
print("BN, One layer Selu with 64 nodes, relu 9 nodes and Softmax")
print("Red")
prediction_red = cross_val_predict(neural_network, X_red, Y_red, cv=cv) 
print(classification_report(Y_red, prediction_red))
print("White")
prediction_white = cross_val_predict(neural_network, X_white, Y_white, cv=cv) 
print(classification_report(Y_white, prediction_white))
print("RSME red")
print(sqrt(np.mean((Y_red-prediction_red)*(Y_red-prediction_red))))
print('RSME white')
print(sqrt(np.mean((Y_white-prediction_white)*(Y_white-prediction_white))))

BN, One layer Selu with 64 nodes, relu 9 nodes and Softmax
Red
              precision    recall  f1-score   support

           1       0.68      0.81      0.74       744
           2       0.56      0.52      0.54       638
           3       0.58      0.30      0.40       217

   micro avg       0.63      0.63      0.63      1599
   macro avg       0.61      0.55      0.56      1599
weighted avg       0.62      0.63      0.61      1599

White
              precision    recall  f1-score   support

           1       0.66      0.65      0.65      1640
           2       0.56      0.64      0.60      2198
           3       0.59      0.43      0.50      1060

   micro avg       0.60      0.60      0.60      4898
   macro avg       0.60      0.57      0.58      4898
weighted avg       0.60      0.60      0.59      4898

RSME red
0.9533028948172204
RSME white
0.9992914657944089


In [16]:
print("BN, One layer Relu with 64 nodes, relu 9 nodes, relu 6 nodes and Softmax")
print("Red")
prediction_red = cross_val_predict(neural_network, X_red, Y_red, cv=cv) 
print(classification_report(Y_red, prediction_red))
print("White")
prediction_white = cross_val_predict(neural_network, X_white, Y_white, cv=cv) 
print(classification_report(Y_white, prediction_white))
print("RSME red")
print(sqrt(np.mean((Y_red-prediction_red)*(Y_red-prediction_red))))
print('RSME white')
print(sqrt(np.mean((Y_white-prediction_white)*(Y_white-prediction_white))))

BN, One layer Relu with 64 nodes, relu 9 nodes, relu 6 nodes and Softmax
Red
              precision    recall  f1-score   support

           1       0.66      0.78      0.71       744
           2       0.51      0.52      0.52       638
           3       0.49      0.15      0.23       217

   micro avg       0.59      0.59      0.59      1599
   macro avg       0.55      0.48      0.49      1599
weighted avg       0.58      0.59      0.57      1599

White
              precision    recall  f1-score   support

           1       0.67      0.65      0.66      1640
           2       0.57      0.64      0.60      2198
           3       0.58      0.44      0.50      1060

   micro avg       0.60      0.60      0.60      4898
   macro avg       0.61      0.58      0.59      4898
weighted avg       0.60      0.60      0.60      4898

RSME red
0.9263366737294821
RSME white
1.0011794846167397


## Feature Selection

In [21]:
### Remove highly correlated variables ###
red_data_selected = deepcopy(X_red)
red_data_selected.pop("free sulfur dioxide") ## low importances, thus removed
red_data_selected.pop("pH") # low importances, thus removed
red_data_selected.pop("residual sugar") # low importances, thus removed
red_data_selected.pop("chlorides") # low importances, thus removed
red_data_selected.pop("citric acid") # # low importances, thus removed
red_data_selected.pop("fixed acidity") # # low importances, thus removed
#red_data_selected.pop("density") # Highly correlated with alcohol, thus removed

#red_data_selected.pop("total sulfur dioxide")
#red_data_selected.pop("sulphates")
#red_data_selected.pop("volatile acidity")
#red_data_selected.pop("alcohol") # kept as most important predictor



              precision    recall  f1-score   support

           1       0.69      0.80      0.74       744
           2       0.55      0.57      0.56       638
           3       0.62      0.23      0.34       217

   micro avg       0.63      0.63      0.63      1599
   macro avg       0.62      0.53      0.55      1599
weighted avg       0.63      0.63      0.62      1599

RSME red
0.9311887338809622


In [9]:

### Remove unimportant variables ###
white_data_selected = deepcopy(X_white)

#white_data_selected.pop("fixed acidity") # Highly correlated with pH and low importance, thus removed
white_data_selected.pop("sulphates") # low importance, thus removed
white_data_selected.pop("chlorides") # low importances, thus removed
white_data_selected.pop("citric acid") # low importances, thus removed
white_data_selected.pop("pH") # low importances, thus removed

#white_data_selected.pop("density") # Highly correlated with alcohol and residual sugar, thus removed
#white_data_selected.pop("total sulfur dioxide") #Higly correlated with free sulphur dioxide, thus removed, because other correlations are higher than off free total dioxide
#white_data_selected.pop("residual sugar") # correlated with alcohol and free sulfur dioxide, thus removed

#white_data_selected.pop("alcohol") # Most important predictor, thus kept
#white_data_selected.pop("free sulfur dioxide") #kept as important predictor
#white_data_selected.pop("volatile acidity") # #kept as important predictor


              precision    recall  f1-score   support

           1       0.65      0.64      0.65      1640
           2       0.55      0.65      0.59      2198
           3       0.57      0.37      0.45      1060

   micro avg       0.58      0.58      0.58      4898
   macro avg       0.59      0.55      0.56      4898
weighted avg       0.59      0.58      0.58      4898

RSME white
0.9864558999677373


In [ ]:
print("BN, One layer Relu with 64 nodes, relu 9 nodes, relu 6 nodes and Softmax")
print("Red")
prediction_red = cross_val_predict(neural_network, X_red, Y_red, cv=cv) 
print(classification_report(Y_red, prediction_red))
print("White")
prediction_white = cross_val_predict(neural_network, X_white, Y_white, cv=cv) 
print(classification_report(Y_white, prediction_white))
print("RSME red")
print(sqrt(np.mean((Y_red-prediction_red)*(Y_red-prediction_red))))
print('RSME white')
print(sqrt(np.mean((Y_white-prediction_white)*(Y_white-prediction_white))))